# Lab 1 - News Categorization

### 문자를 Vector로 – One-hot Encoding
- 하나의 단어를 Vector의 Index로 인식, 단어 존재시 1 없으면 0

### Bag of words
- 단어별로 인덱스를 부여해서, 한 문장(또는 문서)의 단어의 개수를 Vector로 표현

### Cosine distance(Cosine similarity)
- 두 점 사이의 각도

## Data set
- 축구와 야구 선수들의 영문 기사를 분류해보자!

## Process
- 파일을 불러오기
- 파일을 읽어서 단어사전 (corpus) 만들기
- 단어별로 Index 만들기
- 만들어진 인덱스로 문서별로 Bag of words vector 생성
- 비교하고자 하는 문서 비교하기
- 얼마나 맞는지 측정하기

In [7]:
import os

# txt 파일 이름 가져오는 함수
def get_file_list(dir_name):
    return os.listdir(dir_name) # cmd의 "dir" 명령어 실행

# 함수 실행 예시
dir_name = "news_data"
file_list = get_file_list(dir_name)
file_list = [os.path.join(dir_name, file_name) for file_name in file_list] # os.path.join(): 각 os에 맞게 상대 경로 설정해줌("/"(mac,linux) or "\"(window))
file_list

['news_data\\1_Dae-Ho Lee walk-off homer gives Mariners 4-2 win over Rangers.txt',
 'news_data\\1_Korean First Baseman Dae-Ho Lee Becomes Free Agent, Interested In MLB Deal.txt',
 'news_data\\1_Lee Dae-ho Announces MLB Aspirations.txt',
 'news_data\\1_Lee Dae-ho to Start Spring Training in Arizona.txt',
 'news_data\\1_Lee Dae-ho wins MVP in Japan Series.txt',
 "news_data\\1_Mariners' Lee Dae-ho belts a walk-off homer.txt",
 'news_data\\1_Mariners’ Lee Dae-ho gets 1st two-hit game, double.txt',
 'news_data\\1_MLB Team Interested In Dae-Ho Lee.txt',
 "news_data\\1_Seattle Mariners' Newest Signing Dae-Ho Lee Could Become Fan Favorite.txt",
 'news_data\\1_SoftBank Hawks Hope to Renew Contract with Lee Dae-ho.txt',
 'news_data\\2_Dodgers left with questions after latest Hyun-Jin Ryu setback.txt',
 'news_data\\2_Dodgers left-hander Hyun-Jin Ryu expects to be ready for start of season.txt',
 'news_data\\2_Dodgers unsure when Hyun-Jin Ryu will throw another bullpen session.txt',
 'news_data\\2

In [14]:
# 파일별로 내용 읽는 함수
def get_conetents(file_list):
    y_class = []
    X_text = []
    class_dict = {1: "0", 2: "0", 3:"0", 4:"0", 5:"1", 6:"1", 7:"1", 8:"1"} # 0:야구, 1:축구

    for file_name in file_list:
        try:
            f = open(file_name, "r",  encoding="cp949") # window: cp949,   mac or linux: utf-8
            category = int(file_name.split(os.sep)[1].split("_")[0])  # os.sep: "/"(mac, linux), "\"(window)
            y_class.append(class_dict[category])
            X_text.append(f.read())
            f.close()
        except UnicodeDecodeError as e:
            print(e)
            print(file_name)
    return X_text, y_class

# # 함수 실행 예시
# dir_name = "news_data"
# file_list = get_file_list(dir_name)
# file_list = [os.path.join(dir_name, file_name) for file_name in file_list]

# X_text, y_class = get_conetents(file_list)
# X_text

In [16]:
# 의미없는 문장부호 등은 제거하는 함수
def get_cleaned_text(text):
    import re
    text = re.sub('\W+','', text.lower() ) # 정규표현식으로 특수문자 제거, 소문자로 변경 => ex) I'm -> im
    return text

# Corpus 만들기 + 단어별 index 생성하는 함수
def get_corpus_dict(text):
    text = [sentence.split() for sentence in text]  # 공백으로 split
    clenad_words = [get_cleaned_text(word) for words in text for word in words]  # get_cleaned_text()로 문장부호 제거

    from collections import OrderedDict
    corpus_dict = OrderedDict()
    for i, v in enumerate(set(clenad_words)):  # set으로 하는 이유: 중복 제거하기 위해!!
        corpus_dict[v] = i  # 단어 순서별로 index를 부여
    return corpus_dict

In [21]:
# 문서별로 Bag of words vector 생성
def get_count_vector(text, corpus):
    text = [sentence.split() for sentence in text]
    word_number_list = [[corpus[get_cleaned_text(word)] for word in words] for words in text]
    X_vector = [[0 for _ in range(len(corpus))] for x in range(len(text))]

    for i, text in enumerate(word_number_list):
        for word_number in text:
            X_vector[i][word_number] += 1
    return X_vector

In [20]:
# cosine similarity로 비교하는 함수
import math
def get_cosine_similarity(v1,v2):
    "compute cosine similarity of v1 to v2: (v1 dot v2)/{||v1||*||v2||)"
    sumxx, sumxy, sumyy = 0, 0, 0
    for i in range(len(v1)):
        x = v1[i]; y = v2[i]
        sumxx += x*x
        sumyy += y*y
        sumxy += x*y
    return sumxy/math.sqrt(sumxx*sumyy)

In [17]:
# 비교결과 정리하기
def get_similarity_score(X_vector, source):
    source_vector = X_vector[source]
    similarity_list = []
    for target_vector in X_vector:
        similarity_list.append(
            get_cosine_similarity(source_vector, target_vector))
    return similarity_list

def get_top_n_similarity_news(similarity_score, n):
    import operator
    x = {i:v for i, v in enumerate(similarity_score)}
    sorted_x = sorted(x.items(), key=operator.itemgetter(1))

    return list(reversed(sorted_x))[1:n+1]

In [18]:
# 성능 측정하기
def get_accuracy(similarity_list, y_class, source_news):
    source_class = y_class[source_news]

    return sum([source_class == y_class[i[0]] for i in similarity_list]) / len(similarity_list)

In [22]:
if __name__ == "__main__":
    dir_name = "news_data"
    file_list = get_file_list(dir_name)
    file_list = [os.path.join(dir_name, file_name) for file_name in file_list]

    X_text, y_class = get_conetents(file_list)

    corpus = get_corpus_dict(X_text)
    print("Number of words : {0}".format(len(corpus)))
    X_vector = get_count_vector(X_text, corpus)
    source_number = 10

    result = []

    for i in range(80):
        source_number = i

        similarity_score = get_similarity_score(X_vector, source_number)
        similarity_news = get_top_n_similarity_news(similarity_score, 10)
        accuracy_score = get_accuracy(similarity_news, y_class, source_number)
        result.append(accuracy_score)
    print(sum(result) / 80)

Number of words : 4024
0.6950000000000001
